

---



---
# Otimização de hiperparâmetros utilizando a classe GridSearchCV do scikit-learn

Este notebook mostra:


* Como usar modelos Keras no scikit-learn empregando a classe [KerasClassifier](https://www.tensorflow.org/api_docs/python/tf/keras/wrappers/scikit_learn/KerasClassifier) (ou [KerasRegressor](https://www.tensorflow.org/api_docs/python/tf/keras/wrappers/scikit_learn/KerasRegressor) para tarefas de regressão)
* Como otimizar hiperparâmetros de um modelo de aprendizado profundo utilizando Keras e a classe [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) (*cross-validated grid-search*) do scikit-learn.

O notebook foi aplicado na tarefa de classifcação do conjunto de dados de [predição política](https://https://github.com/malegopc/AM2PUCPOC/tree/main/Datasets/Predicao_Politica). 



---



---



## Classes específicas

A classe KerasClassifier (ou KerasRegressor) permitem a utilização de modelos Keras em scikit-learn por meio do encapsulamento dos modelos.

A classe GridSearch realiza um pesquisa (combinação) exaustiva de valores de hiperparâmetros para um modelo. Os hiperparâmetros do modelo são otimizados por validação cruzada sobre uma grade (*grid*) de parâmetros fornecida.

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

## Leitura dos dados

In [ ]:
import pandas as pd

# Clone do repositório de dados do GitHub
!git clone https://github.com/malegopc/DSBD
# cria vetor com os rótulos de cada atributo
nomes_atributos = ['Partido', 'Crianças deficientes', 'Compartilhamento de custos do projeto de água',
                    'Resolução de adoção do orçamento', 'Congelamento de honorários médicos',
                    'Ajuda El Salvador', 'Grupos religiosos nas escolas',
                    'Proibição de teste anti-satélite', 'Ajuda a contras nicaraguenses',
                    'Míssil MX', 'Imigração', 'Corte de corporação de combustíveis Syn',
                    'Gastos com educação', ' Super fundo direto', 'Crime',
                    'Exportações isentas de impostos', 'Exportação África do Sul']

# lê arquivo de dados, atribue NaN para dados faltantes e rótulos em cada coluna
dados_votacao = pd.read_csv('/content/DSBD/Datasets/Predicao_Politica/votes_data.txt', na_values=['?'], names = nomes_atributos)
# imprime as 5 primeiras linha dos dados montados
dados_votacao.head()

Cloning into 'DSBD'...
remote: Enumerating objects: 549, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 549 (delta 39), reused 71 (delta 28), pack-reused 457
Receiving objects: 100% (549/549), 10.07 MiB | 16.44 MiB/s, done.
Resolving deltas: 100% (247/247), done.


,Partido,Crianças deficientes,Compartilhamento de custos do projeto de água,Resolução de adoção do orçamento,Congelamento de honorários médicos,Ajuda El Salvador,Grupos religiosos nas escolas,Proibição de teste anti-satélite,Ajuda a contras nicaraguenses,Míssil MX,Imigração,Corte de corporação de combustíveis Syn,Gastos com educação,Super fundo direto,Crime,Exportações isentas de impostos,Exportação África do Sul
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


## Eliminação de dados ausentes 

In [ ]:
# Remove dados faltantes na própria variável 
dados_votacao.dropna(inplace=True)
dados_votacao.describe()

,Partido,Crianças deficientes,Compartilhamento de custos do projeto de água,Resolução de adoção do orçamento,Congelamento de honorários médicos,Ajuda El Salvador,Grupos religiosos nas escolas,Proibição de teste anti-satélite,Ajuda a contras nicaraguenses,Míssil MX,Imigração,Corte de corporação de combustíveis Syn,Gastos com educação,Super fundo direto,Crime,Exportações isentas de impostos,Exportação África do Sul
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


## Codificação de dados categóricos

In [ ]:
dados_votacao.replace(('y', 'n'), (1, 0), inplace=True)
dados_votacao.replace(('democrat', 'republican'), (1, 0), inplace=True)
dados_votacao.head()

,Partido,Crianças deficientes,Compartilhamento de custos do projeto de água,Resolução de adoção do orçamento,Congelamento de honorários médicos,Ajuda El Salvador,Grupos religiosos nas escolas,Proibição de teste anti-satélite,Ajuda a contras nicaraguenses,Míssil MX,Imigração,Corte de corporação de combustíveis Syn,Gastos com educação,Super fundo direto,Crime,Exportações isentas de impostos,Exportação África do Sul
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


## Separa os atributos (recursos) e a variável target (alvo)

In [ ]:
X = dados_votacao[nomes_atributos].drop('Partido', axis=1).values
y = dados_votacao['Partido'].values
print(X.shape)
print(y.shape)

(232, 16)
(232,)


## Número de nerônios das camadas ocultas

Busca os melhores números de neurônios das duas camadas ocultas.

In [ ]:
# função que cria o modelo Keras
def cria_model(units1, units2):
  # cria modelo	
  model = Sequential()
  model.add(Dense(units=units1, kernel_initializer = 'he_uniform', activation='relu',input_dim=16))
  model.add(Dense(units=units2, kernel_initializer = 'he_uniform', activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  # Compila o modelo
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
# cria e encapsula o modelo por meio do Keras_Classifier
model = KerasClassifier(build_fn=cria_model, verbose=0)
# busca os melhores parâmetros para batch_size e epochs
units1 = [8, 16, 32, 64]
units2 = [4, 8, 16, 32]
param_grid = dict(units1=units1, units2=units2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
# Melhor resultado (melhores parâmetros)
print("Melhor resultado: %f utilizando os parâmetros %s" % (grid_results.best_score_, grid_results.best_params_))
# Análise dos resultados dos testes
medias = grid_results.cv_results_['mean_test_score']
desvios = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for media, desvio, param in zip(medias, desvios, params):
    print("%f (%f) utilizando: %r" % (media, desvio, param))

Melhor resultado: 0.801976 utilizando os parâmetros {'units1': 32, 'units2': 16}
0.585137 (0.176221) utilizando: {'units1': 8, 'units2': 4}
0.421967 (0.142283) utilizando: {'units1': 8, 'units2': 8}
0.448551 (0.248043) utilizando: {'units1': 8, 'units2': 16}
0.404762 (0.121173) utilizando: {'units1': 8, 'units2': 32}
0.539294 (0.082178) utilizando: {'units1': 16, 'units2': 4}
0.573815 (0.200799) utilizando: {'units1': 16, 'units2': 8}
0.555056 (0.166020) utilizando: {'units1': 16, 'units2': 16}
0.634033 (0.158302) utilizando: {'units1': 16, 'units2': 32}
0.603730 (0.165618) utilizando: {'units1': 32, 'units2': 4}
0.430847 (0.167215) utilizando: {'units1': 32, 'units2': 8}
0.801976 (0.067198) utilizando: {'units1': 32, 'units2': 16}
0.589966 (0.102307) utilizando: {'units1': 32, 'units2': 32}
0.602842 (0.112766) utilizando: {'units1': 64, 'units2': 4}
0.534299 (0.030603) utilizando: {'units1': 64, 'units2': 8}
0.766733 (0.091244) utilizando: {'units1': 64, 'units2': 16}
0.491009 (0.0711

## Tamanho do lote (batch_size) e número de épocas (epochs)

Busca os melhores valores para batch_size e número de épocas.

In [ ]:
# função que cria o modelo Keras
def cria_model():
  # cria modelo	
  model = Sequential()
  model.add(Dense(units=32, kernel_initializer = 'he_uniform', activation='relu',input_dim=16))
  model.add(Dense(units=16, kernel_initializer = 'he_uniform', activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  # Compila o modelo
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
# cria e encapsula o modelo por meio do Keras_Classifier
model = KerasClassifier(build_fn=cria_model, verbose=0)
# busca os melhores parâmetros para batch_size e epochs
batch_size = [4, 8, 16, 32]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
# Melhor resultado (melhores parâmetros)
print("Melhor resultado: %f utilizando os parâmetros %s" % (grid_results.best_score_, grid_results.best_params_))
# Análise dos resultados dos testes
medias = grid_results.cv_results_['mean_test_score']
desvios = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for media, desvio, param in zip(medias, desvios, params):
    print("%f (%f) utilizando: %r" % (media, desvio, param))

Melhor resultado: 0.961205 utilizando os parâmetros {'batch_size': 4, 'epochs': 50}
0.948274 (0.021210) utilizando: {'batch_size': 4, 'epochs': 10}
0.961205 (0.000235) utilizando: {'batch_size': 4, 'epochs': 50}
0.956876 (0.006243) utilizando: {'batch_size': 4, 'epochs': 100}
0.948274 (0.000314) utilizando: {'batch_size': 8, 'epochs': 10}
0.956821 (0.012401) utilizando: {'batch_size': 8, 'epochs': 50}
0.956876 (0.006243) utilizando: {'batch_size': 8, 'epochs': 100}
0.896492 (0.018647) utilizando: {'batch_size': 16, 'epochs': 10}
0.961150 (0.010740) utilizando: {'batch_size': 16, 'epochs': 50}
0.952547 (0.006358) utilizando: {'batch_size': 16, 'epochs': 100}
0.849095 (0.012698) utilizando: {'batch_size': 32, 'epochs': 10}
0.939560 (0.016420) utilizando: {'batch_size': 32, 'epochs': 50}
0.943834 (0.022226) utilizando: {'batch_size': 32, 'epochs': 100}


## Otimizador

Busca o melhor otimizador para treinar a rede.

In [ ]:
# função que cria o modelo Keras
def cria_model(optimizer):
  # cria modelo	
  model = Sequential()
  model.add(Dense(units=32, kernel_initializer = 'he_uniform', activation='relu',input_dim=16))
  model.add(Dense(units=16, kernel_initializer = 'he_uniform', activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  # Compila o modelo
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model
# cria e encapsula o modelo por meio do Keras_Classifier
model = KerasClassifier(build_fn=cria_model, verbose=0)
# busca os melhores parâmetros para o otimizador
optimizer = ['SGD','RMSprop','Adam','Adadelta','Adagrad','Adamax','Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
# Melhor resultado (melhores parâmetros)
print("Melhor resultado: %f utilizando os parâmetros %s" % (grid_results.best_score_, grid_results.best_params_))
# Análise dos resultados dos testes
medias = grid_results.cv_results_['mean_test_score']
desvios = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for media, desvio, param in zip(medias, desvios, params):
    print("%f (%f) utilizando: %r" % (media, desvio, param))

Melhor resultado: 0.740759 utilizando os parâmetros {'optimizer': 'RMSprop'}
0.560051 (0.075656) utilizando: {'optimizer': 'SGD'}
0.740759 (0.135909) utilizando: {'optimizer': 'RMSprop'}
0.370352 (0.105804) utilizando: {'optimizer': 'Adam'}
0.396825 (0.044896) utilizando: {'optimizer': 'Adadelta'}
0.638695 (0.125268) utilizando: {'optimizer': 'Adagrad'}
0.646243 (0.054960) utilizando: {'optimizer': 'Adamax'}
0.581585 (0.066407) utilizando: {'optimizer': 'Nadam'}


## Número de neurônios, batch_size, épocas e otimizador

*Obs: aproximadamente 3 horas na GPU do Colab*.

In [ ]:
# função que cria o modelo Keras
def cria_model(units1, units2, optimizer):
  # cria modelo	
  model = Sequential()
  model.add(Dense(units=units1, kernel_initializer = 'he_uniform', activation='relu',input_dim=16))
  model.add(Dense(units=units2, kernel_initializer = 'he_uniform', activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  # Compila o modelo
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model
# cria e encapsula o modelo por meio do Keras_Classifier
model = KerasClassifier(build_fn=cria_model, verbose=0)
# busca os melhores parâmetros para número de neurônios, batch_size, épocas e otimizador
units1 = [8, 16, 32, 64]
units2 = [4, 8, 16, 32]
batch_size = [4, 8, 16, 32]
epochs = [10, 50, 100]
optimizer = ['SGD','RMSprop','Adam','Adadelta','Adagrad','Adamax','Nadam']
param_grid = dict(units1=units1, units2=units2, batch_size=batch_size, epochs=epochs, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
# Melhor resultado (melhores parâmetros)
print("Melhor resultado: %f utilizando os parâmetros %s" % (grid_results.best_score_, grid_results.best_params_))
# Análise dos resultados dos testes
medias = grid_results.cv_results_['mean_test_score']
desvios = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for media, desvio, param in zip(medias, desvios, params):
    print("%f (%f) utilizando: %r" % (media, desvio, param))

Melhor resultado: 0.974081 utilizando os parâmetros {'batch_size': 4, 'epochs': 50, 'optimizer': 'SGD', 'units1': 16, 'units2': 32}
0.861916 (0.025116) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 8, 'units2': 4}
0.887890 (0.022256) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 8, 'units2': 8}
0.922300 (0.018680) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 8, 'units2': 16}
0.913697 (0.022348) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 8, 'units2': 32}
0.796981 (0.169371) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 16, 'units2': 4}
0.922411 (0.010614) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 16, 'units2': 8}
0.913864 (0.015728) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 16, 'units2': 16}
0.896437 (0.018837) with: {'batch_size': 4, 'epochs': 10, 'optimizer': 'SGD', 'units1': 16, 'units2': 32}
0.896437 (0.028366) with: {'